In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys

In [3]:
from rmab.simulator import RMABSimulator, random_valid_transition, random_valid_transition_round_down, synthetic_transition_small_window
from rmab.uc_whittle import UCWhittle, UCWhittleFixed, UCWhittleMatch, NormPlusMatch
from rmab.ucw_value import UCWhittle_value
from rmab.baselines import optimal_policy, random_policy, WIQL, optimal_match_policy
from rmab.fr_dynamics import get_all_transitions
from rmab.utils import get_save_path, delete_duplicate_results


In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [13]:
if is_jupyter: 
    seed        = 42
    n_arms      = 8
    budget      = 3 
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 30
    episode_len = 20 
    n_epochs    = 10
    save_name = 'combined_lamb'
    match_prob = 0.5
    save_with_date = False 
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=8)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=20)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=30)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=10)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--save_name',      '-n', help='save name', type=str, default='combined_lamb')
    parser.add_argument('--match_prob',      '-m', help='match probability', type=float, default=0.5)
    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    save_name   = args.save_name 
    save_with_date = args.use_date 
    match_prob = args.match_prob 



In [14]:
n_states = 2
n_actions = 2

In [15]:
all_population_size = 100 # number of random arms to generate
all_transitions = get_all_transitions(all_population_size)

In [16]:
all_transitions.shape

(100, 2, 2, 2)

In [17]:
all_features = np.arange(all_population_size)

In [18]:
np.random.seed(seed)
random.seed(seed)
simulator = RMABSimulator(all_population_size, all_features, all_transitions,
            n_arms, episode_len, n_epochs, n_episodes, budget, number_states=n_states, reward_style='match',match_probability=match_prob)

acting should always be good! 0.000 < 0.044
acting should always be good! 0.000 < 0.162
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [83 53]


In [19]:
rewards_by_lamb = {}
std_by_lamb = {}
active_rates_by_lamb = {}

In [20]:
for lamb in [0.25,0.5,0.75,0.9,1,1.1,1.25,1.5,2]:
    np.random.seed(seed)
    random.seed(seed)
    rewards_combined = NormPlusMatch(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',lamb=lamb)
    combined_active_rate = simulator.total_active/(rewards_combined.size*n_arms)

    rewards_by_lamb[lamb] = np.mean(rewards_combined)
    std_by_lamb[lamb] = np.std(rewards_combined) 
    active_rates_by_lamb[lamb] = combined_active_rate 

solving UCWhittle using method: UCB
first state [0 1]
instance 0, ep 1, state [1 1]
instance 0, ep 2, state [1 1]
instance 0, ep 3, state [1 1]
instance 0, ep 4, state [0 0]
instance 0, ep 5, state [0 1]
instance 0, ep 6, state [1 1]
instance 0, ep 7, state [0 1]
instance 0, ep 8, state [0 1]
instance 0, ep 9, state [1 1]
instance 0, ep 10, state [0 0]
instance 0, ep 11, state [0 1]
instance 0, ep 12, state [0 0]
instance 0, ep 13, state [0 1]
instance 0, ep 14, state [0 1]
instance 0, ep 15, state [1 0]
instance 0, ep 16, state [1 1]
instance 0, ep 17, state [0 1]
instance 0, ep 18, state [1 1]
instance 0, ep 19, state [1 1]
instance 0, ep 20, state [0 0]
instance 0, ep 21, state [0 0]
instance 0, ep 22, state [0 1]
instance 0, ep 23, state [0 0]
instance 0, ep 24, state [0 1]
instance 0, ep 25, state [0 1]
instance 0, ep 26, state [0 0]
instance 0, ep 27, state [0 0]
instance 0, ep 28, state [1 0]
instance 0, ep 29, state [1 0]
solving UCWhittle using method: UCB
first state [0 1]
in

In [21]:
data = {
    'mean_reward': rewards_by_lamb, 
    'std_reward': std_by_lamb,
    'active_rate': active_rates_by_lamb, 
    'parameters': 
        {'seed'      : seed,
        'n_arms'    : n_arms,
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'match_prob': match_prob} 
}

In [23]:
save_path = get_save_path('matching',save_name,seed,use_date=save_with_date)

In [24]:
delete_duplicate_results('matching',save_name,data)

In [25]:
json.dump(data,open('../results/'+save_path,'w'))